In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline
from stable_baselines3 import PPO, A2C
from IPython.display import clear_output

import glob

KeyboardInterrupt: 

In [ ]:
import sys
sys.path.append(r"C:\Users\fidel\OneDrive\Documentos\Mestrado\code")
from RL_trading_envs import StockWithVolHold, process_data
from analysis import *

In [ ]:
raw_env  = StockWithVolHold

In [ ]:
path_data = r"D:\Datasets\TwoSigmaDataSetBr_2022_03_04\Papertrade_br\models\MlpVarianceSwap\trained_model\model_MlpVarianceSwap_2022-03-03\df_test_mestrado.pkl"
df_valid, df_test = process_data(path_data)

In [ ]:
#Subset of assets to symplify and make faster
assets = [0, 38, 42, 45, 47, 48, 49, 51, 53, 95]
# assets = [53, 95]
df_valid.query("asset_identifier.isin(@assets) & date < 100", inplace = True)
# df_valid.loc[:, 'target_return'] = df_valid.loc[:, 'target_return'] - df_valid.loc[:, 'target_return'].mean()
df_valid.reset_index(drop = True, inplace = True)
df_valid.shape

In [ ]:
m = np.random.normal(size = (100, 100))
w = np.random.normal(size = (100, ))

In [ ]:
%%time
for i in range(100):
    a = m[i, :] * w
#     a = w.T @ m @ w

In [ ]:
model = PPO.load('models_save/model_ppo_21', env)
env = raw_env(df_valid)
done = False
obs = env.reset()
list_actions = []

In [ ]:
env.state_to_features(True)

In [ ]:
rm = np.random.normal(size = 100)
ra = rm + np.random.normal(size = 100)
rb = rm + np.random.normal(size = 100)
rm = rm + np.random.normal(size = 100)

In [133]:
np.cov(ra, rm)

array([[2.33217205, 0.93167425],
       [0.93167425, 1.87566723]])

In [134]:
np.cov(rb, rm)

array([[2.3412681 , 1.18689917],
       [1.18689917, 1.87566723]])

In [135]:
np.cov(ra + rb, rm)

array([[7.17023537, 2.11857342],
       [2.11857342, 1.87566723]])

In [136]:
0.93167425 + 1.18689917

2.11857342

In [139]:
np.corrcoef(ra+rb, rm)

array([[1.        , 0.57769541],
       [0.57769541, 1.        ]])

In [140]:
np.corrcoef(ra, rm)

array([[1.        , 0.44545712],
       [0.44545712, 1.        ]])

In [141]:
np.corrcoef(rb, rm)

array([[1.        , 0.56638319],
       [0.56638319, 1.        ]])

In [144]:
(0.44545712*ra.std() + 0.56638319*rb.std())/(ra+rb).std()

0.5776954101649441

In [153]:
ra.std()**2

2.3193898587605104

In [154]:
ra.var()

2.3193898587605104

In [120]:
env.hidden_state

{53: [0.02484299833802011, 0.029195644568623075, nan],
 95: [0.01624068009994628, 0.03064397666355116, -0.5]}

In [127]:
action = np.array([0])
obs, reward, done, _ = env.step(action)

0.0817094477816558


In [128]:
reward

-0.11208414253460394

In [113]:
env.state_to_features(True)

{'portfolio_volatility': 0.021728104955728193,
 'stock_volatility': 0.030141055606223478,
 'stock_prediction': 0.00021350919269025326,
 'available_notional': 0.0,
 'stock_notional': -0.4757201955680461}

In [47]:
env.hidden_state

{53: [0.49188482761383057, 0.0322944681809501, nan],
 95: [0.012428343296051025, 0.03115384915128543, -0.5]}

In [29]:
#Loop through env
for i in range(3):#not done:
    action, _ = model.predict(obs, deterministic = True)
    obs, reward, done, _ = env.step(action)
    list_actions.append(action.item())


0.6093204225183958
0.9656904808672576
NEW DAY: 1
1.9781593326476732


In [30]:
env.state_to_features(True)

{'portfolio_volatility': 0.01978159332647673,
 'stock_volatility': 0.0322944681809501,
 'stock_prediction': -0.0006703828694298863,
 'available_notional': 0.20768312354527652,
 'stock_notional': 0.1879977667364514}

In [35]:
env.hidden_state

{53: [0.1879977667364514, 0.0322944681809501, nan],
 95: [0.6043191097182721, 0.03115384915128543, 0.35812127590179443]}

In [31]:
obs

array([ 1.97815933,  3.22944682, -6.70382869,  0.20768312,  1.87997767])

In [34]:
action = np.array(-0.187/0.5)

In [ ]:
obs, reward, done, _ = env.step(action)

In [25]:
list_actions

[0.3821987509727478, 0.4804531931877136, 0.7162425518035889]

In [ ]:
#Extract information from hidden state memory
list_hidden_state = []
for day, hidden_state in env.hidden_state_memory.items():
    hidden_state = pd.DataFrame(hidden_state).T.reset_index()
    hidden_state.columns = ['asset_identifier', 'notional', 'vol', 'action']
    hidden_state.loc[:, 'date'] = day
    list_hidden_state.append(hidden_state)
hidden_state = pd.concat(list_hidden_state)